# 🧪 LLM.int8() From Scratch: Outlier-Aware Quantization (2022)

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/chronology/llm_int8_demo.ipynb)

## 📖 The Theory: Vector-wise Quantization & Outliers

Large Language Models (LLMs) exhibit a unique property: as they scale, certain feature dimensions in the activations develop **emergent outliers**. These outliers have magnitudes significantly larger than the rest of the values (often $>6.0$ while the mean is $<0.1$).

### The Challenge
Standard 8-bit quantization (INT8) maps a range $[min, max]$ to $[-128, 127]$. If a single outlier is $60.0$ and most values are $0.1$, the quantization scale becomes massive, crushing all the precision of the $0.1$ values into zero.

### The LLM.int8() Solution
LLM.int8() uses a two-part strategy:
1.  **Vector-wise Quantization**: Each row/column of the weight matrix and activation matrix is quantized with its own scale factor.
2.  **Outlier Handling**: Dimensions that contain outliers ($> \text{threshold}$) are extracted and multiplied in high precision (**FP16/FP32**). The rest are multiplied in **INT8**.

$$Y = X_{int8} \cdot W_{int8} + X_{outlier} \cdot W_{outlier}$$

---

In [ ]:
import torch
import time
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 1. Load Model & Tokenizer
model_id = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_id)
model = GPT2LMHeadModel.from_pretrained(model_id).eval()

def print_size_in_mb(tensor, name):
    size = tensor.numel() * tensor.element_size() / (1024**2)
    print(f"{name} Size: {size:.2f} MB")

weight = model.transformer.h[0].attn.c_attn.weight.data
print_size_in_mb(weight, "Original FP32 Weight")

## 🛠️ Implementation: Manual Vector-wise Quantization
Instead of using black-box libraries, we implement the core math manually to demonstrate the efficiency gains.

In [ ]:
def llm_int8_matmul(X, W, threshold=6.0):
    """
    A from-scratch simulation of outlier-aware matmul.
    """
    # 1. Identify Outlier Dimensions
    X = X.float()
    W = W.float()
    col_max = torch.max(torch.abs(X), dim=0)[0]
    outlier_idx = torch.where(col_max > threshold)[0]
    non_outlier_idx = torch.where(col_max <= threshold)[0]
    
    # 2. Extract and multiply Outliers in high precision
    X_outlier = X[:, outlier_idx]
    W_outlier = W[outlier_idx, :]
    outlier_res = torch.matmul(X_outlier, W_outlier)
    
    # 3. Quantize and multiply others in INT8 simulation
    X_non = X[:, non_outlier_idx]
    W_non = W[non_outlier_idx, :]
    
    if X_non.shape[1] > 0:
        # Vector-wise quant for X (row units)
        x_scale = torch.max(torch.abs(X_non), dim=1, keepdim=True)[0] / 127
        X_q = torch.round(X_non / (x_scale + 1e-8)).to(torch.int8)
        
        # Vector-wise quant for W (col units)
        w_scale = torch.max(torch.abs(W_non), dim=0, keepdim=True)[0] / 127
        W_q = torch.round(W_non / (w_scale + 1e-8)).to(torch.int8)
        
        # Matmul simulation (using Torch's matmul on float32 converted from int8)
        int8_res = torch.matmul(X_q.to(torch.float32), W_q.to(torch.float32))
        int8_res = int8_res * (x_scale * w_scale)
    else:
        int8_res = 0
    
    return int8_res + outlier_res

# Test with synthetic data containing outliers
X = torch.randn(16, 512)
X[:, 7] *= 20.0 # Dimension 7 is an outlier
W = torch.randn(512, 1024)

start = time.time()
fp32_res = torch.matmul(X, W)
time_fp32 = time.time() - start

start = time.time()
int8_res = llm_int8_matmul(X, W)
time_int8 = time.time() - start

mse = torch.mean((fp32_res - int8_res)**2)
print(f"Reconstruction MSE (with outliers): {mse:.8f}")
print(f"FP32 Time: {time_fp32:.4f}s")
print(f"INT8 Simulation Time: {time_int8:.4f}s")
print(f"\nWeight storage effectively reduced to 8-bit (4x compression simulation)!")